# Libs

In [1]:
from deap import base, creator, tools, algorithms
import pandas as pd
import random

# Disciplinas

In [2]:
disciplinas = pd.read_excel('disciplinas.xlsx')
disciplinas['Disciplina'] = disciplinas['Disciplina'].str.upper()
disciplinas = disciplinas.groupby('Disciplina')['Créditos'].sum()
disciplinas = [(disciplina, creditos) for disciplina, creditos in disciplinas.items()]

# Professores

In [3]:
professores = pd.read_excel('professores.xlsx')
professores['Preferências'] = professores[['Disciplinas', 'Disciplinas.1', 'Disciplinas.2']].bfill(axis=1).iloc[:, 0]
professores['Preferências'] = professores['Preferências'].str.upper()
professores['Preferências'] = professores['Preferências'].str.split(';')
professores['Nome'] = professores['Nome de usuário'].str.split('@').str[0]
professores = professores.loc[:,['Nome', 'Preferências', 'CH MAX']].dropna()
professores = professores[['Nome', 'Preferências', 'CH MAX']].to_dict(orient='records')

# Alocacao

In [4]:
# Quantidade de disciplinas e professores
num_disciplinas = len(disciplinas)
num_professores = len(professores)

# Criar tipos de fitness e indivíduo
creator.create("FitnessMulti", base.Fitness, weights=(1, -1))  # Maximizar preferências e minimizar desvio de carga
creator.create("Individual", list, fitness=creator.FitnessMulti)

# Configuração do toolbox
toolbox = base.Toolbox()

# Registro de função para inicializar um indivíduo com alocação aleatória
toolbox.register("indices", random.randint, 0, num_professores - 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.indices, n=num_disciplinas)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Função de avaliação do indivíduo
def avaliar(individual):
    carga_horaria = [0] * num_professores  # Armazena a carga horária de cada professor
    preferencias = 0  # Contador de preferências atendidas
    desvio_ch = 0  # Soma do desvio em relação à carga horária máxima

    # Alocação das disciplinas e cálculo das cargas horárias e preferências atendidas
    for i, professor_idx in enumerate(individual):
        carga_horaria[professor_idx] += disciplinas[i][1]
        if disciplinas[i][0] in professores[professor_idx]['Preferências']:
            preferencias += 1

    # Calcula o desvio em relação à carga horária máxima para cada professor
    for i, ch in enumerate(carga_horaria):
        desvio_ch += abs(ch - professores[i]['CH MAX']) * 2

    return 100 * preferencias / num_disciplinas, desvio_ch  # Retorna percentual de preferências e desvio de carga horária

# Registro dos operadores genéticos
toolbox.register("mate", tools.cxUniform, indpb=0.5)#toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutUniformInt, low=0, up=num_professores - 1, indpb=0.2)
toolbox.register("select", tools.selNSGA2)
toolbox.register("evaluate", avaliar)

# Configuração das estatísticas do algoritmo
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg_preferencia", lambda fits: sum(fit[0] for fit in fits) / len(fits))  # Média de preferências atendidas
stats.register("avg_desvio", lambda fits: sum(fit[1] for fit in fits) / len(fits))  # Média do excedente de carga horária

# Parâmetros do algoritmo genético
populacao = toolbox.population(n=100000)
algorithms.eaMuPlusLambda(populacao, toolbox, mu=20000, lambda_=60000, cxpb=0.7, mutpb=0.1, ngen=150, stats=stats, verbose=True)

# Melhor resultado
melhor_individuo = tools.selBest(populacao, k=1)[0]

gen	nevals	avg_pref	avg_excedente
0  	100000	21.5506 	197.839      
1  	48032 	24.9013 	139.112      
2  	47989 	26.4497 	123.176      
3  	47971 	28.5046 	113.801      
4  	48159 	30.0376 	105.631      
5  	47990 	31.4832 	99.3334      
6  	48014 	33.3603 	95.3468      
7  	47875 	34.5591 	91.0812      
8  	47959 	36.3666 	88.1684      
9  	48055 	37.361  	84.1598      
10 	47825 	38.8423 	81.888       
11 	48176 	39.5503 	78.4378      
12 	48006 	40.5487 	75.9064      
13 	47947 	41.4696 	73.7258      
14 	47915 	42.4762 	71.6908      
15 	47962 	42.3488 	68.3228      
16 	47881 	44.1232 	67.238       
17 	48090 	44.4109 	64.666       
18 	47946 	46.0242 	63.7464      
19 	47802 	46.1094 	61.372       
20 	48030 	46.4729 	60.5162      
21 	47957 	42.122  	52.4108      
22 	48118 	48.5634 	57.9316      
23 	47993 	41.0369 	46.1536      
24 	48126 	36.4921 	40.8236      
25 	48083 	42.1078 	44.3364      
26 	48103 	45.9533 	44.8536      
27 	47985 	38.7216 	36.5728      
28 	47760 	39.

# Resultado

In [5]:
# Listar professores com suas disciplinas e carga horária alocadas
alocacao_final = {professor['Nome']: {'disciplinas': [], 'carga_horaria': 0} for professor in professores}

# Distribuir as disciplinas entre os professores conforme o melhor indivíduo
for i, professor_idx in enumerate(melhor_individuo):
    professor_nome = professores[professor_idx]['Nome']
    alocacao_final[professor_nome]['disciplinas'].append(disciplinas[i][0])
    alocacao_final[professor_nome]['carga_horaria'] += disciplinas[i][1]
    
# Transformar o dicionário em uma lista de tuplas para criar o DataFrame
dados_alocacao = []
for professor, dados in alocacao_final.items():
    disciplinas_alocadas = ', '.join(dados['disciplinas'])  # Juntar as disciplinas em uma única string
    carga_horaria_total = dados['carga_horaria']
    dados_alocacao.append((professor, disciplinas_alocadas, carga_horaria_total))

# Criar o DataFrame
df_alocacao = pd.DataFrame(dados_alocacao, columns=['Professor', 'Disciplinas Alocadas', 'Carga Horária Total (h.a)'])

# Salvar o DataFrame
df_alocacao.to_excel('alocacao.xlsx')

# Exibir o DataFrame
df_alocacao

,Professor,Disciplinas Alocadas,Carga Horária Total (h.a)
0,adriano.franca,"INTERNET, PROTOCOLOS DE COMUNICAÇÃO, REDES DE ...",12
1,ricson.santana,"DESENVOLVIMENTO WEB, INFORMÁTICA BÁSICA, PROGR...",16
2,valdir.silva,"ADMINISTRAÇÃO DE SISTEMAS OPERACIONAIS, PRÁTIC...",17
3,delano.oliveira,"LINGUAGEM E TÉCNICAS DE PROGRAMAÇÃO I, PROJETO...",12
4,victor.cavalcanti,"BANCO DE DADOS II, ENGENHARIA DE SOFTWARE, EST...",16
5,thiago.bezerra,"ARQUITETURA DE COMPUTADORES, METODOLOGIA CIENT...",18
6,saulomarques.morais,"CABEAMENTO ESTRUTURADO, LEGISLAÇÃO EM INFORMÁT...",22
7,leonardo.lima,"PROJETO DE DESENVOLVIMENTO DE SISTEMAS WEB I, ...",19
8,tiago.lima,"INTELIGÊNCIA ARTIFICIAL, LÓGICA DE PROGRAMAÇÃO...",17
9,david.ribeiro,"INTERAÇÃO HUMANO-COMPUTADOR, INTERFACE HOMEM-M...",14
